In [60]:
import pandas as pd
import numpy as np

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 머신러닝 모델 및 도구
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [74]:
# make file 
# 엑셀 파일 불러오기
df_comm = pd.read_csv("seoul_commerical.csv", encoding='EUC-KR')
df_comm.rename(columns={'행정동_코드_명': '행정동'}, inplace=True)

# 해당 연분기의 데이터 선택
df_comm['기준_년분기_코드'] = df_comm['기준_년분기_코드'].astype(str)

# 필요한 열만 선택
df_filtered = df_comm[['행정동', '기준_년분기_코드', '여가_문화_지출_총금액']]

# 피벗 테이블 생성: 행은 '행정동', 열은 '기준_년분기_코드', 값은 '여가_문화_지출_총금액'
pivot_df = df_filtered.pivot_table(index='행정동',
                                    columns='기준_년분기_코드',
                                    values='여가_문화_지출_총금액',
                                    aggfunc='sum')
pivot_df = pivot_df / 10000
pivot_df.to_csv('seoul_quarter_commerical.csv', encoding='utf-8-sig')
pivot_df.rename(columns={'기준_년분기_코드 행정동': '행정동'}, inplace=True)
pivot_df

기준_년분기_코드,20191,20192,20193,20194,20201,20202,20203,20204,20211,20212,...,20223,20224,20231,20232,20233,20234,20241,20242,20243,20244
행정동,,,,,,,,,,,,,,,,,,,,,
가락1동,976.7,2742.5,5694.0,6619.9,5972.4,6075.7,6965.0,6100.9,7732.2,11018.2,...,21706.0,19690.9,23704.4,27040.3,26873.9,25111.6,8140.3,23059.5,8857.3,20776.8
가락2동,24211.5,28670.5,31174.1,26322.9,22098.0,26554.3,24056.0,20526.6,18830.4,29244.3,...,30464.8,27267.6,29444.2,32625.2,34085.6,29707.9,10294.5,31240.9,11523.5,29320.8
가락본동,80798.5,91568.5,87795.3,86659.8,67906.0,79581.1,60011.3,66875.0,62812.2,81037.1,...,83875.3,95502.4,83331.1,83209.2,79947.1,89586.4,27053.3,85077.5,27520.1,81740.6
가리봉동,3827.9,5010.0,5007.8,4413.4,4449.7,4455.0,4424.8,4513.2,4405.1,13274.7,...,7469.2,7189.0,7322.3,6954.0,6548.6,6422.4,1888.1,5750.6,2074.7,6398.2
가산동,260854.5,296081.2,277126.2,283551.3,200981.8,235046.3,231374.4,290192.3,272056.3,352717.7,...,396961.2,486975.0,644102.0,673625.0,769633.2,887366.7,351024.5,954394.8,354094.6,1073202.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
효창동,5747.6,6618.2,7262.1,8126.6,4519.6,6966.1,5166.4,4298.1,4850.0,5933.8,...,9012.7,7440.1,7571.7,7024.7,7505.1,4706.2,2021.5,5735.5,1384.6,3977.3
후암동,3234.9,3145.8,3577.2,3578.3,5075.8,17677.6,12930.8,12147.6,14079.5,13907.2,...,15412.7,14553.7,15631.0,15020.5,15449.0,14838.2,2280.7,7518.6,3295.8,6704.8
휘경1동,19841.7,17889.2,16620.1,16791.3,14988.0,13030.4,11705.8,10332.0,11210.8,14953.0,...,20754.5,26723.1,30261.2,29192.9,30359.8,27163.8,10112.8,28275.6,8734.3,27547.5


In [106]:

# 1. 반려동물 시설 데이터 불러오기 및 전처리
df_pet = pd.read_csv("../facility/pet_facilities_with_area2.csv")
df_pet_filtered = df_pet[['DONG', '시설수']].copy()
df_pet_filtered['시설수'] = df_pet_filtered['시설수'].round(2)

# 2. 행정동별 시설 수 집계
df_pet_grouped = df_pet_filtered.groupby('DONG', as_index=False).sum()
df_pet_grouped.rename(columns={'DONG': '행정동'}, inplace=True)

# 3. 여가·문화 지출 데이터 피벗 테이블 생성
# (기존에 생성된 pivot_df를 사용한다고 가정)
# pivot_df = df_filtered.pivot_table(index='행정동',
#                                     columns='기준_년분기_코드',
#                                     values='여가_문화_지출_총금액',
#                                     aggfunc='sum')
# pivot_df = pivot_df / 10000

# 4. 피벗 테이블과 반려동물 시설 데이터 병합
merged_df = pivot_df.reset_index().merge(df_pet_grouped, on='행정동', how='left')

# 5. 행정동 면적 데이터 불러오기 및 전처리
df_area = pd.read_csv('../facility/dong_area.csv')
df_area.columns = df_area.columns.str.strip()  # 열 이름 공백 제거
df_area_filtered = df_area[['행정동', '면적']].copy()
df_area_filtered['면적'] = df_area_filtered['면적'].round(2)

# 6. 병합된 데이터에 면적 정보 추가
merged_df = merged_df.merge(df_area_filtered, on='행정동', how='left')

# 7. 최종 데이터 저장
merged_df.to_csv('seoul_leisure_pet_facilities.csv', index=False, encoding='utf-8-sig')
merged_df

,행정동,20191,20192,20193,20194,20201,20202,20203,20204,20211,...,20231,20232,20233,20234,20241,20242,20243,20244,시설수,면적
0,가락1동,976.7,2742.5,5694.0,6619.9,5972.4,6075.7,6965.0,6100.9,7732.2,...,23704.4,27040.3,26873.9,25111.6,8140.3,23059.5,8857.3,20776.8,NaN,1.34
1,가락2동,24211.5,28670.5,31174.1,26322.9,22098.0,26554.3,24056.0,20526.6,18830.4,...,29444.2,32625.2,34085.6,29707.9,10294.5,31240.9,11523.5,29320.8,NaN,0.96
2,가락본동,80798.5,91568.5,87795.3,86659.8,67906.0,79581.1,60011.3,66875.0,62812.2,...,83331.1,83209.2,79947.1,89586.4,27053.3,85077.5,27520.1,81740.6,1089.0,1.13
3,가리봉동,3827.9,5010.0,5007.8,4413.4,4449.7,4455.0,4424.8,4513.2,4405.1,...,7322.3,6954.0,6548.6,6422.4,1888.1,5750.6,2074.7,6398.2,9.0,0.40
4,가산동,260854.5,296081.2,277126.2,283551.3,200981.8,235046.3,231374.4,290192.3,272056.3,...,644102.0,673625.0,769633.2,887366.7,351024.5,954394.8,354094.6,1073202.9,361.0,2.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,효창동,5747.6,6618.2,7262.1,8126.6,4519.6,6966.1,5166.4,4298.1,4850.0,...,7571.7,7024.7,7505.1,4706.2,2021.5,5735.5,1384.6,3977.3,4.0,0.44
421,후암동,3234.9,3145.8,3577.2,3578.3,5075.8,17677.6,12930.8,12147.6,14079.5,...,15631.0,15020.5,15449.0,14838.2,2280.7,7518.6,3295.8,6704.8,16.0,0.86
422,휘경1동,19841.7,17889.2,16620.1,16791.3,14988.0,13030.4,11705.8,10332.0,11210.8,...,30261.2,29192.9,30359.8,27163.8,10112.8,28275.6,8734.3,27547.5,49.0,0.63
423,휘경2동,10747.5,11883.8,11683.4,13491.4,13793.7,12149.3,11239.8,21211.2,19423.0,...,26105.2,27393.7,29666.3,25923.0,8686.9,28600.8,10569.3,29147.7,NaN,1.05
